# Analysing reproducibility of benchmark runs

Comparing 3 runs of the Operetta drug profiling dataset (see input data https://zenodo.org/records/18506825)

In [63]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment


In [2]:
# Data loading
df1 = pd.read_csv("/Users/joel/Library/CloudStorage/Dropbox/Joel/BioVisionCenter/Fractal/Fractal_Paper/Analysis_reproducibility/regionprops_filtered_public.csv")
df2 = pd.read_csv("/Users/joel/Library/CloudStorage/Dropbox/Joel/BioVisionCenter/Fractal/Fractal_Paper/Analysis_reproducibility/regionprops_filtered_scloud.csv")
df3 = pd.read_csv("/Users/joel/Library/CloudStorage/Dropbox/Joel/BioVisionCenter/Fractal/Fractal_Paper/Analysis_reproducibility/regionprops_filtered_scluster.csv")

In [18]:
# Cell counts and counts of classified cells per well
print(f"Number of cells in df1: {len(df1)}")
print(f"Number of cells in df2: {len(df2)}")
print(f"Number of cells in df3: {len(df3)}")

Number of cells in df1: 1595788
Number of cells in df2: 1595788
Number of cells in df3: 1595788


In [19]:
print(f"Classification categories:{df1['BVC_test_data_CyQ_classifier_prediction'].unique()}")

Classification categories:['MSC' 'live leukemia' 'dead']


In [14]:
WELL_KEYS = ["row", "column"]
CLASS_COL = "BVC_test_data_CyQ_classifier_prediction"

def per_well_class_counts(df):
    return (
        df.groupby(WELL_KEYS)[CLASS_COL]
        .value_counts(dropna=False)
        .rename("count")
        .reset_index()
    )

def calculate_per_well_class_count_differences(df_r1, df_r2):
    cw1 = per_well_class_counts(df_r1)
    cw2 = per_well_class_counts(df_r2)

    p1 = cw1.pivot_table(
        index=WELL_KEYS,
        columns=CLASS_COL,
        values="count",
        fill_value=0,
    )

    p2 = cw2.pivot_table(
        index=WELL_KEYS,
        columns=CLASS_COL,
        values="count",
        fill_value=0,
    )

    p1a, p2a = p1.align(p2, fill_value=0)
    count_diff = p1a - p2a

    wells_with_count_diff = count_diff.any(axis=1)
    print(f"Number of wells: {len(wells_with_count_diff)}")

    return wells_with_count_diff.sum()

In [15]:
print(
    "Wells with differing classification counts between R1 and R2:"
    f"{calculate_per_well_class_count_differences(df1, df2)}"
)
print(
    "Wells with differing classification counts between R1 and R3:"
    f"{calculate_per_well_class_count_differences(df1, df3)}"
)
print(
    "Wells with differing classification counts between R2 and R3:"
    f"{calculate_per_well_class_count_differences(df2, df3)}"
)

Number of wells: 240
Wells with differing classification counts between R1 and R2:0
Number of wells: 240
Wells with differing classification counts between R1 and R3:0
Number of wells: 240
Wells with differing classification counts between R2 and R3:0


## Single cell measurement comparisons

In [25]:
# Check that columns match
print(sorted(df1.columns) == sorted(df2.columns))
print(sorted(df1.columns) == sorted(df3.columns))
print(sorted(df2.columns) == sorted(df3.columns))

True
True
True


In [ ]:
# Exact image_url will vary depending on deployment. 
# Plate names depend on base folder name in a given deployment
# Labels have some permutations due to Stardist segmentation
# BVC_test_data_CyQ_classifier_prediction has already been shown to be equal and is a string column
MEAS_COLS = [
    c for c in df1.columns
    if c not in {"BVC_test_data_CyQ_classifier_prediction", "label", "image_url", "plate_name"}
    and pd.api.types.is_numeric_dtype(df1[c])
]
MEAS_COLS

['area',
 'area_convex',
 'intensity_mean',
 'intensity_max',
 'intensity_min',
 'intensity_std',
 'eccentricity',
 'perimeter',
 'perimeter_crofton',
 'solidity',
 'equivalent_diameter_area',
 'feret_diameter_max',
 'axis_major_length',
 'axis_minor_length',
 'orientation',
 'extent',
 'inertia_tensor_eigvals-0',
 'inertia_tensor_eigvals-1',
 'bbox-0',
 'bbox-1',
 'bbox-2',
 'bbox-3',
 'area_bbox',
 'centroid-0',
 'centroid-1',
 'centroid_weighted-0',
 'centroid_weighted-1',
 'column',
 'path_in_well']

In [50]:
def extract_well(df, well_row, well_col, cols):
    return (
        df[(df["row"] == well_row) & (df["column"] == well_col)]
        [cols]
        .copy()
    )

def canonicalize_measurements(df, cols, decimals=None):
    # Ensure column sorting & optionally rounding to avoid numerical precision issues
    tmp = df[sorted(cols)].copy()
    if decimals is not None:
        tmp = tmp.round(decimals)
    return tmp

def object_hashes(df):
    return pd.util.hash_pandas_object(df, index=False)

def permutation_test_well(
    dfA,
    dfB,
    well_row,
    well_col,
    meas_cols,
    decimals=6
):
    A = extract_well(dfA, well_row, well_col, meas_cols)
    B = extract_well(dfB, well_row, well_col, meas_cols)

    print("Object counts:", len(A), len(B))

    A = canonicalize_measurements(A, meas_cols, decimals)
    B = canonicalize_measurements(B, meas_cols, decimals)

    hA = object_hashes(A).sort_values().reset_index(drop=True)
    hB = object_hashes(B).sort_values().reset_index(drop=True)

    identical = hA.equals(hB)

    return {
        "identical_multiset": identical,
        "n_objects_A": len(A),
        "n_objects_B": len(B),
        "n_unique_hash_A": hA.nunique(),
        "n_unique_hash_B": hB.nunique(),
        "n_hash_diff": (~hA.isin(hB)).sum() + (~hB.isin(hA)).sum()
    }

def split_matches(dfA, dfB, well_row, well_col, cols, decimals=6):
    A = extract_well(dfA, well_row, well_col, cols)
    B = extract_well(dfB, well_row, well_col, cols)

    A = canonicalize_measurements(A, cols, decimals)
    B = canonicalize_measurements(B, cols, decimals)

    hA = object_hashes(A)
    hB = object_hashes(B)

    A = A.copy()
    B = B.copy()
    A["_hash"] = hA.to_numpy()
    B["_hash"] = hB.to_numpy()

    # Collision diagnostics
    nA, nB = len(A), len(B)
    uA, uB = A["_hash"].nunique(), B["_hash"].nunique()
    collA, collB = nA - uA, nB - uB

    mask_A = hA.isin(hB)
    mask_B = hB.isin(hA)

    matched_A = A[mask_A]
    matched_B = B[mask_B]
 
    unmatched_A = A[~mask_A]
    unmatched_B = B[~mask_B]

    return matched_A, matched_B, unmatched_A, unmatched_B, collA, collB

def calc_mismatches_across_wells(df_r1, df_r2, wells, meas_cols, decimals=6):
    mismatched_cells = {}
    mismatch_counter = 0
    for well_row, well_col in wells:
        matched_A, matched_B, unmatched_A, unmatched_B, collA, collB = split_matches(
            df_r1,
            df_r2,
            well_row=well_row,
            well_col=well_col,
            cols=meas_cols,
            decimals=decimals,
        )
        assert collA == 0, f"Hash collisions in A for well {well_row}{well_col}"
        assert collB == 0, f"Hash collisions in B for well {well_row}{well_col}"
        total_objects = len(matched_A) + len(unmatched_A)
        fraction_matched = len(matched_A) / total_objects
        print(
            f"Matched {len(matched_A)} out of {total_objects} objects "
            f"({fraction_matched:.2%}) in well {well_row}{well_col}"
        )
        if len(unmatched_A) > 0 or len(unmatched_B) > 0:
            mismatch_counter += total_objects - len(matched_A)
            mismatched_cells[f"{well_row}{well_col}"] = {
                "unmatched_A": unmatched_A,
                "unmatched_B": unmatched_B
            }
            print(f"Unmatched cells in {well_row}{well_col}:")
            print(f"  Unmatched A: {len(unmatched_A)} objects")
            print(f"  Unmatched B: {len(unmatched_B)} objects")
    
    print(f"Total mismatched cells across wells: {mismatch_counter}")
    return mismatched_cells, mismatch_counter

In [33]:
result = permutation_test_well(
    df1,
    df3,
    well_row="C",
    well_col=3,
    meas_cols=MEAS_COLS,
    decimals=6
)

result

Object counts: 9601 9601


{'identical_multiset': False,
 'n_objects_A': 9601,
 'n_objects_B': 9601,
 'n_unique_hash_A': 9601,
 'n_unique_hash_B': 9601,
 'n_hash_diff': np.int64(2)}

In [35]:
matched_A, matched_B, unmatched_A, unmatched_B = split_matches(
    df1,
    df2,
    well_row="C",
    well_col=3,
    cols=MEAS_COLS,
    decimals=6
)
total_objects = len(matched_A) + len(unmatched_A)
fraction_matched = len(matched_A) / total_objects
print(f"Matched {len(matched_A)} out of {total_objects} objects ({fraction_matched:.2%}) in well C3")

Matched 9600 out of 9601 objects (99.99%) in well C3


In [52]:
wells = list(df1[["row", "column"]].drop_duplicates().itertuples(index=False, name=None))

mismatched_cells_12, mismatch_counter_12 = calc_mismatches_across_wells(df1, df2, wells, MEAS_COLS, decimals=6)
mismatched_cells_13, mismatch_counter_13 = calc_mismatches_across_wells(df1, df3, wells, MEAS_COLS, decimals=6)
mismatched_cells_23, mismatch_counter_23 = calc_mismatches_across_wells(df2, df3, wells, MEAS_COLS, decimals=6)

print(mismatch_counter_12)
print(mismatch_counter_13)
print(mismatch_counter_23)

Matched 9644 out of 9644 objects (100.00%) in well C10
Matched 9232 out of 9232 objects (100.00%) in well C11
Matched 9681 out of 9681 objects (100.00%) in well C12
Matched 10002 out of 10002 objects (100.00%) in well C13
Matched 9949 out of 9949 objects (100.00%) in well C14
Matched 9056 out of 9056 objects (100.00%) in well C15
Matched 8700 out of 8700 objects (100.00%) in well C16
Matched 8224 out of 8224 objects (100.00%) in well C17
Matched 9709 out of 9709 objects (100.00%) in well C18
Matched 9982 out of 9982 objects (100.00%) in well C19
Matched 9142 out of 9142 objects (100.00%) in well C20
Matched 8846 out of 8846 objects (100.00%) in well C21
Matched 6787 out of 6787 objects (100.00%) in well C22
Matched 9600 out of 9601 objects (99.99%) in well C3
Unmatched cells in C3:
  Unmatched A: 1 objects
  Unmatched B: 1 objects
Matched 9776 out of 9776 objects (100.00%) in well C4
Matched 10155 out of 10156 objects (99.99%) in well C5
Unmatched cells in C5:
  Unmatched A: 1 objects


In [54]:
def verify_unmatched_symmetry(mismatched_cells):
    summary = []

    for well, d in mismatched_cells.items():
        nA = len(d["unmatched_A"])
        nB = len(d["unmatched_B"])

        summary.append({
            "well": well,
            "n_unmatched_A": nA,
            "n_unmatched_B": nB,
            "symmetric": nA == nB
        })

    return pd.DataFrame(summary)

In [62]:
print(
    "Matching number of unmatched cells in both replicates for R1 vs R2: "
    f"{verify_unmatched_symmetry(mismatched_cells_12)["symmetric"].all()}"
)
print(
    "Matching number of unmatched cells in both replicates for R1 vs R3: "
    f"{verify_unmatched_symmetry(mismatched_cells_13)["symmetric"].all()}"
)
print(
    "Matching number of unmatched cells in both replicates for R2 vs R3: "
    f"{verify_unmatched_symmetry(mismatched_cells_23)["symmetric"].all()}"
)

Matching number of unmatched cells in both replicates for R1 vs R2: True
Matching number of unmatched cells in both replicates for R1 vs R3: True
Matching number of unmatched cells in both replicates for R2 vs R3: True


In [94]:
def match_unmatched_cells(dfA, dfB, cols):
    num_cols = [c for c in cols if pd.api.types.is_numeric_dtype(dfA[c])]

    XA = dfA[num_cols].to_numpy(dtype="float64")
    XB = dfB[num_cols].to_numpy(dtype="float64")

    # Standardize features
    scale = np.nanstd(np.vstack([XA, XB]), axis=0)
    scale[scale == 0] = 1.0

    XA = XA / scale
    XB = XB / scale

    # Pairwise distance matrix
    dist = np.sqrt(((XA[:, None, :] - XB[None, :, :]) ** 2).sum(axis=2))

    row_ind, col_ind = linear_sum_assignment(dist)

    return row_ind, col_ind, dist[row_ind, col_ind]

def compare_unmatched_cells(dfA, dfB, cols):
    rows_A, rows_B, _ = match_unmatched_cells(dfA, dfB, cols)

    diffs = []

    for iA, iB in zip(rows_A, rows_B):
        a = dfA.iloc[iA]
        b = dfB.iloc[iB]

        entry = {}

        for c in cols:
            if not pd.api.types.is_numeric_dtype(dfA[c]):
                continue

            absdiff = abs(a[c] - b[c])

            rel = np.nan
            if a[c] != 0:
                rel = absdiff / abs(a[c])

            entry[f"absdiff__{c}"] = absdiff
            entry[f"reldiff__{c}"] = rel

        diffs.append(entry)

    return pd.DataFrame(diffs)


In [ ]:
# def match_unmatched_cells_by_centroid(
#     dfA: pd.DataFrame,
#     dfB: pd.DataFrame,
#     c0: str = "centroid-0",
#     c1: str = "centroid-1",
#     max_dist: float | None = None,   # optional gating in pixels
# ):
#     """
#     Promixity-based pairing of unmatched cells based on centroid coordinates.

#     - Uses Hungarian assignment to minimize total Euclidean distance.
#     - If max_dist is set, pairs beyond max_dist are excluded (returned as unassigned).
#     """

#     if len(dfA) == 0 or len(dfB) == 0:
#         return {
#             "pairs": [],                 # list of (iA, iB, dist)
#             "unassigned_A": list(range(len(dfA))),
#             "unassigned_B": list(range(len(dfB))),
#         }

#     XA = dfA[[c0, c1]].to_numpy(dtype="float64")
#     XB = dfB[[c0, c1]].to_numpy(dtype="float64")

#     # Pairwise Euclidean distances
#     dist = np.sqrt(((XA[:, None, :] - XB[None, :, :]) ** 2).sum(axis=2))

#     row_ind, col_ind = linear_sum_assignment(dist)

#     pairs = []
#     unassigned_A = set(range(len(dfA)))
#     unassigned_B = set(range(len(dfB)))

#     for iA, iB in zip(row_ind, col_ind):
#         d = float(dist[iA, iB])
#         if (max_dist is None) or (d <= max_dist):
#             pairs.append((iA, iB, d))
#             unassigned_A.discard(iA)
#             unassigned_B.discard(iB)

#     return {
#         "pairs": pairs,
#         "unassigned_A": sorted(unassigned_A),
#         "unassigned_B": sorted(unassigned_B),
#     }

# def diffs_for_centroid_pairs(dfA, dfB, pairs, cols, rel_eps=0.0):
#     num_cols = [c for c in cols if pd.api.types.is_numeric_dtype(dfA[c])]
#     rows = []

#     for iA, iB, d in pairs:
#         a = dfA.iloc[iA]
#         b = dfB.iloc[iB]
#         row = {"A_index": a.name, "B_index": b.name, "centroid_dist": d}

#         for c in num_cols:
#             av = float(a[c]) if pd.notna(a[c]) else np.nan
#             bv = float(b[c]) if pd.notna(b[c]) else np.nan
#             absdiff = abs(av - bv) if (np.isfinite(av) and np.isfinite(bv)) else np.nan

#             if np.isnan(av):
#                 reldiff = np.nan
#             else:
#                 denom = abs(av)
#                 if denom == 0:
#                     reldiff = np.nan if rel_eps == 0.0 else absdiff / rel_eps
#                 else:
#                     reldiff = absdiff / denom

#             row[f"absdiff__{c}"] = absdiff
#             row[f"reldiff__{c}"] = reldiff

#         rows.append(row)

#     return pd.DataFrame(rows)

# all_diffs_12 = []
# for well, d in mismatched_cells_12.items():

#     dfA = d["unmatched_A"]
#     dfB = d["unmatched_B"]

#     match = match_unmatched_cells_by_centroid(dfA, dfB, max_dist=5.0)  # e.g. 5 px gate
#     match["pairs"], match["unassigned_A"], match["unassigned_B"]
#     diff_df = diffs_for_centroid_pairs(dfA, dfB, match["pairs"], MEAS_COLS)
#     diff_df.sort_values("centroid_dist", ascending=False).head()
#     all_diffs_12.append(diff_df)
    
# all_diffs_12 = pd.concat(all_diffs_12, ignore_index=True)
# all_diffs_12

In [100]:
all_diffs_12 = []

for well, d in mismatched_cells_12.items():

    dfA = d["unmatched_A"]
    dfB = d["unmatched_B"]

    diff_df = compare_unmatched_cells(dfA, dfB, MEAS_COLS)
    diff_df["well"] = well

    all_diffs_12.append(diff_df)

all_diffs_12 = pd.concat(all_diffs_12, ignore_index=True)
all_diffs_12.to_csv("unmatched_cell_differences_R1_vs_R2.csv", index=False)

In [101]:
all_diffs_13 = []

for well, d in mismatched_cells_13.items():

    dfA = d["unmatched_A"]
    dfB = d["unmatched_B"]

    diff_df = compare_unmatched_cells(dfA, dfB, MEAS_COLS)
    diff_df["well"] = well

    all_diffs_13.append(diff_df)

all_diffs_13 = pd.concat(all_diffs_13, ignore_index=True)
all_diffs_13.to_csv("unmatched_cell_differences_R1_vs_R3.csv", index=False)

In [102]:
all_diffs_23 = []

for well, d in mismatched_cells_23.items():

    dfA = d["unmatched_A"]
    dfB = d["unmatched_B"]

    diff_df = compare_unmatched_cells(dfA, dfB, MEAS_COLS)
    diff_df["well"] = well

    all_diffs_23.append(diff_df)

all_diffs_23 = pd.concat(all_diffs_23, ignore_index=True)
all_diffs_23.to_csv("unmatched_cell_differences_R2_vs_R3.csv", index=False)

In [72]:
def mismatches_per_well(mismatched_cells):
    rows = []

    for well, d in mismatched_cells.items():
        nA = len(d["unmatched_A"])
        nB = len(d["unmatched_B"])

        rows.append({
            "well": well,
            "n_unmatched_A": nA,
            "n_unmatched_B": nB,
            "n_unmatched": max(nA, nB),
            "symmetric": nA == nB
        })

    return (
        pd.DataFrame(rows)
        .sort_values("n_unmatched", ascending=False)
        .reset_index(drop=True)
    )

In [73]:
overview = mismatches_per_well(mismatched_cells_12)
overview

,well,n_unmatched_A,n_unmatched_B,n_unmatched,symmetric
0,F4,2,2,2,True
1,C3,1,1,1,True
2,J19,1,1,1,True
3,M4,1,1,1,True
4,M10,1,1,1,True
5,L6,1,1,1,True
6,K7,1,1,1,True
7,K12,1,1,1,True
8,J9,1,1,1,True
9,J7,1,1,1,True


In [81]:
mismatched_cells_12["F4"]['unmatched_A'][['centroid-0', 'centroid-1']]

,centroid-0,centroid-1
633160,470.363636,543.509091
633161,469.788462,550.557692


In [80]:
mismatched_cells_12["F4"]['unmatched_B'][['centroid-0', 'centroid-1']]

,centroid-0,centroid-1
633160,469.693548,549.935484
633161,470.622222,542.800000


In [92]:
mismatched_cells_12["F4"]['unmatched_A']

,area,area_bbox,area_convex,axis_major_length,axis_minor_length,bbox-0,bbox-1,bbox-2,bbox-3,centroid-0,...,intensity_max,intensity_mean,intensity_min,intensity_std,orientation,path_in_well,perimeter,perimeter_crofton,solidity,_hash
633160,55.0,72.0,58.0,10.025907,7.010420,467.0,540.0,475.0,549.0,470.363636,...,9657.0,5257.581818,2138.0,1927.323445,-0.810943,0,25.899495,27.235778,0.948276,5992628226609854595
633161,52.0,72.0,57.0,9.557717,7.202834,466.0,547.0,475.0,555.0,469.788462,...,8814.0,5793.365385,2355.0,1654.235229,-0.182263,0,26.142136,27.465816,0.912281,14692529198288602755


In [93]:
mismatched_cells_12["F4"]['unmatched_B']

,area,area_bbox,area_convex,axis_major_length,axis_minor_length,bbox-0,bbox-1,bbox-2,bbox-3,centroid-0,...,intensity_max,intensity_mean,intensity_min,intensity_std,orientation,path_in_well,perimeter,perimeter_crofton,solidity,_hash
633160,62.0,81.0,63.0,8.994622,8.790308,466.0,546.0,475.0,555.0,469.693548,...,8814.0,5673.709677,2355.0,1604.082165,-0.418224,0,26.142136,27.465816,0.984127,18082681673879738294
633161,45.0,56.0,50.0,9.181190,6.441537,467.0,540.0,475.0,547.0,470.622222,...,9657.0,5303.377778,2138.0,2061.096303,-0.379252,0,24.278175,25.895019,0.900000,363005745853287607
